In [1]:
import numpy as np
import pyvisa
import time

In [ ]:
class RP7972A():
    """Esta clase pide y mide los datos de la fuente, para luego ser procesados"""
    def __init__(self, curr_initialFreq, curr_finalFreq, curr_amplitude, curr_Freqs, points_per_decade, rp_USB=None):
        """Define variables de entrada como variables globales en la clase"""
        self.curr_initialFreq = curr_initialFreq
        self.curr_finalFreq = curr_finalFreq
        self.curr_amplitude = curr_amplitude
        self.curr_Freqs = curr_Freqs #an array of the frequencies used for the sweep
        self.points_per_decade = points_per_decade # the points per decade to generate the frequencies for the sweep

        self.rp_USB=rp_USB
        # FOR INITIALIZATION OF POWER SOURCE (RP7972A)
        rm = pyvisa.ResourceManager()
        devices = rm.list_resources()

        print("Searching for devices...")
        if devices:
            for device in devices:
                try:
                    my_instrument = rm.open_resource(str(device))
                    device_ID=my_instrument.query('*IDN?')
                    if  "RP797" in device_ID:#if the RP7972 is found assign it to rp_USB
                        self.rp_USB=my_instrument
                        print("Found and connected to: "+device_ID)
                        break
                    else:#close other devices
                        my_instrument.close()
                except:
                    print("Error: No response from found device")
            if not self.rp_USB:
                print("No RP7972 power supply was found.")
                rm.close()
        else:
            print("No devices were found")

        
    def generate_arb(self, frequency): #TODO update this function to new version found in generate arb v7
        """Para generar las señales de corriente que se envian a la fuente, se tiene que llamar por cada frecuencia, no es sweep"""
        """INFO: service manual, cap 4, Programming an Arbitrary Waveform"""
        arb_cycles, arb_points_per_cycle = self.cycles_per_frequency(frequency)
        arb_points, dwell_time = self.arb_data(frequency, arb_points_per_cycle)
        # Enable arb function
        arb_enable = "CURR:MODE ARB"
        self.scpi_out(arb_enable)
        # Specify arb type
        arb_type = "ARB:FUNC:TYPE CURR"
        self.scpi_out(arb_type)
        # Specify dwell time
        arb_dwell = "ARB:CURR:CDW:DWEL " + str(dwell_time)
        self.scpi_out(arb_dwell)
        # Program arb points
        self.scpi_points_out(arb_points) # with scpi out points function not scipi out
        # Speecify the arb to repeat
        if arb_cycles > 1:
            arb_repeat = "ARB:COUN " + str(arb_cycles)
            self.scpi_out(arb_repeat)
        # Specify the last output
        arb_end = "ARB:END:LAST OFF"
        self.scpi_out(arb_end)
        
    def measure_voltage(self, measure_time, measure_points):
        """Query the device for voltage points using the MEASure:ARRay:VOLTage[:DC]? command"""
        try:
            self.scpi_out(" SENS:SWE:TINT " + str(measure_time) )
            self.scpi_out(" SENS:SWE:POIN " + str(measure_points))
            self.scpi_out("TRIG:ACQ:SOUR TRAN")
            self.scpi_out("INIT:ACQ")
            time.sleep(0.1) #it would be best to query the statur of the ACQ system
            self.scpi_out(" TRIG:ACQ")
            time.sleep(measure_time*measure_points +0.2) #it would be best to query the active measurment status to fetch measurement after it is done
            points = self.rp_USB.query("FETC:ARR:VOLT?")
            return points
        except:
            print("ERROR: no data was recieved")
        #TODO deal with the points returned, separate them and save them in an array that can be accessed by the EIS class
       
    # TOOLBOX
    def sweep_frequencies(self):
        """A partir de las frecuencias inicial y final y la cantidad de sweep points se definen las frecuencias sobre las que se hará el barrido"""
        decade=0#An exponent that iterates in the while
        upper_lim=False
        log_points=(np.logspace(0,1,self.points_per_decade,False))#generates an array of points equally distributed on a log scale from 1 to 10 based on the ppd given
        while not upper_lim:#loops as long as the frequency values calculated don't exceed the final frequency
            frec_array=(self.curr_initialFreq*10**decade)*log_points#applies the log distributed point array to the initial frequency
            for i in frec_array:#goes through every point in the array generated in the previous line
                if i < self.curr_finalFreq:#if the value is less than the final frequency it gets appended to the 
                    self.sweep_frequencies.append(float(i))
                else:
                    upper_lim = True#breaks the while
            decade +=1#ups the exponent for the next decade
        self.curr_Freqs.append(self.curr_finalFreq)#adds the final frequency

    def cycles_per_frequency(self, freq):
        """Switch para definir ciclos por frecuencia, recibe una frequencia saca cantidad de ciclos"""
        if freq <= 2000:
            cycles = 4
            points_per_cycle = 50
        elif 100 <= freq < 1000:
            cycles = 2
            points_per_cycle = 100
        elif 10 <= freq <= 100:
            cycles = 2
            points_per_cycle = 100
        elif 1 <= freq <= 10:
            cycles = 2
            points_per_cycle = 100
        elif 100/1000 <= freq <= 1:
            cycles = 2
            points_per_cycle = 100
        elif 10/1000 <= freq <= 100/1000:
            cycles = 1
            points_per_cycle = 1000
        elif 1/1000 <= freq <= 10/1000:
            cycles = 1
            points_per_cycle = 1000
        return cycles, points_per_cycle
    def arb_data(self, frequency, points_per_cycle):
        """Generates data points for arb generation and dwell time for a frequency that is input.
        Minimum dwell time is 10 us (10*10^-6s)"""
        x = np.linspace(0, 2*np.pi, points_per_cycle)
        sin_points=np.sin(x)
        dwell_time=(1/frequency)/points_per_cycle
        return sin_points, dwell_time
        # Serán 1000 puntos por ciclo máximo, si hay problemas lo reduciremos (dato random)\

    #=============== SCPI communication functions ===================
    def scpi_out(self, command):
        """Outputs an encoded scpi string on serial port"""
        try:
            self.rp_USB.write(str(command),"\n")
        except:
            print("ERROR: Could not send the message: " + str(command))
    def scpi_points_out(self, values):
        """Outputs the command to set the ARB points and the points as scpi values"""
        try:
            self.rp_USB.write_ascii_values('ARB:CURRent:CDWell', values, "f",",","\n")
        except:
            print("ERROR: Could not send array of ascii points")
    def scpi_query(self, query):
        try:
            self.rp_USB.query(str(query))
        except:
            print("ERROR: Could not send query: " + str(query))
    def stop(self):
        try:
            self.rp_USB.write("ABORt:TRANsient","\n")
            self.rp_USB.write("OUTP OFF","\n")
            self.rp_USB.write("ABORt:ACQuire","\n")
            self.rp_USB.write("ABORt:ELOG","\n")
        except:
            print("ERROR: Could not send stop commands")
    #======================== ELOG config (no longer used) ==================================
    def elog_config(self, selector): #maybe we should measure the current as well?
        try:
            """Select the Measurement Function"""
            self.scpi_out("SENS:ELOG:FUNC:CURR ON")
            """ Specify the Integration Period minimum of 102.4 microseconds to a maximum of 60 seconds."""
            self.scpi_out("SENS:ELOG:PER 0.0006")
            """Select the Elog Trigger Source"""
            self.scpi_out(" TRIG:ELOG:SOUR IMM")
            if selector:
                self.scpi_out("FORM[:DATA] REAL")
                self.scpi_out("FORM:BORD NORM")
        except:
            print("ELOG config error")
    def elog_start(self):
        try:
            self.scpi_out("INIT:ELOG")
        except:
            print("ELOG start error")
    def elog_retrieve_data(self, wait, records):
        try:
            time.sleep(wait)
            data=self.rp_USB.query_ascii_values("FETC:ELOG? "+str(records),"f",",",)
        except:
            print("Retrieve data error")
        return data
    def elog_stop(self):
        self.scpi_out("ABOR:ELOG")
    def elog_retrieve_data_binary(self, wait, records):
        try:
            time.sleep(wait)
            data=self.rp_USB.query_binary_values("FETC:ELOG? "+str(records),"f",True)#TODO finish parameters to retrieve binary data
        except:
            print("Retrieve data error")
        return data

         


class EIS():
    """Una vez tengamos los datos, se procesaran con esta clase"""
    